In [27]:
import rasterio
import pandas as pd
from pyproj import Transformer
import glob
import numpy as np

In [53]:
import os
import ntpath
cwd = os.getcwd()
cwd

'C:\\Users\\nimbus\\PycharmProjects\\LandCover\\notebooks'

In [206]:
input_path = "data/landcover/*"

In [222]:
file_paths = glob.glob(input_path)
file_paths

['data/landcover\\B02_10m.tiff',
 'data/landcover\\B03_10m.tiff',
 'data/landcover\\B04_10m.tiff',
 'data/landcover\\B08_10m.tiff',
 'data/landcover\\B11_10m.tiff',
 'data/landcover\\B12_10m.tiff',
 'data/landcover\\ground_truth.tif',
 'data/landcover\\NDBI.tiff',
 'data/landcover\\NDDI.tiff',
 'data/landcover\\NDUI.tiff',
 'data/landcover\\NDVI.tiff',
 'data/landcover\\NDWI.tiff']

In [224]:
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [226]:
def file_name_from_path(path):
    return path_leaf(path).split(".")[0]

In [228]:
for file_path in file_paths:
    print(file_name_from_path(file_path))

B02_10m
B03_10m
B04_10m
B08_10m
B11_10m
B12_10m
ground_truth
NDBI
NDDI
NDUI
NDVI
NDWI


In [230]:
def get_data_frame(file_paths, latlon_crs = 'epsg:4326'):
    df_list = []
    for file_path in file_paths:
        with rasterio.open(file_path) as f:
            zz = f.read(1)
            x = np.linspace(f.bounds.left, f.bounds.right, f.shape[1])
            y = np.linspace(f.bounds.bottom, f.bounds.top, f.shape[0])
        xx, yy = np.meshgrid(x, y)
        df = pd.DataFrame({
            'x': xx.flatten(),
            'y': yy.flatten(),
            'value': zz.flatten(),
        })
        transformer = Transformer.from_crs(f.crs, latlon_crs, always_xy=False)
        df['lat'], df['lon'] = transformer.transform(xx=df.x, yy=df.y)
        df.drop(columns=['x', 'y'], inplace=True)
        df = df[['lat', 'lon', 'value']]
        file_name = file_name_from_path(file_path)
        df = df.rename(columns={"value": file_name})
        df_list.append(df)
    return df_list

In [236]:
df_list =  get_data_frame(file_paths, latlon_crs = 'epsg:4326')
df_list

[            lat        lon  B02_10m
 0     -8.990286  51.785690        0
 1     -8.988815  51.785690        0
 2     -8.987344  51.785690        0
 3     -8.985873  51.785690        0
 4     -8.984402  51.785690        0
 ...         ...        ...      ...
 13627 -8.788750  51.871608        0
 13628 -8.787279  51.871608        0
 13629 -8.785808  51.871608        0
 13630 -8.784337  51.871608        0
 13631 -8.782866  51.871608        0
 
 [13632 rows x 3 columns],
             lat        lon  B03_10m
 0     -8.990286  51.785690        0
 1     -8.988815  51.785690        0
 2     -8.987344  51.785690        0
 3     -8.985873  51.785690        0
 4     -8.984402  51.785690        0
 ...         ...        ...      ...
 13627 -8.788750  51.871608        0
 13628 -8.787279  51.871608        0
 13629 -8.785808  51.871608        0
 13630 -8.784337  51.871608        0
 13631 -8.782866  51.871608        0
 
 [13632 rows x 3 columns],
             lat        lon  B04_10m
 0     -8.990286 

In [238]:
len(df_list[1:])

11

In [240]:
result_df = df_list[0]
for df in df_list[1:]:
    result_df = pd.merge(result_df, df, how="left", on=["lat", "lon"])


In [242]:
result_df

,lat,lon,B02_10m,B03_10m,B04_10m,B08_10m,B11_10m,B12_10m,ground_truth,NDBI,NDDI,NDUI,NDVI,NDWI
0,-8.990286,51.785690,0,0,0,0,0,0,-128,0,0,0,0,0
1,-8.988815,51.785690,0,0,0,0,0,0,-128,0,0,0,0,0
2,-8.987344,51.785690,0,0,0,0,0,0,-128,0,0,0,0,0
3,-8.985873,51.785690,0,0,0,0,0,0,-128,0,0,0,0,0
4,-8.984402,51.785690,0,0,0,0,0,0,-128,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13627,-8.788750,51.871608,0,0,0,0,0,0,-128,0,0,0,0,0
13628,-8.787279,51.871608,0,0,0,0,0,0,-128,0,0,0,0,0
13629,-8.785808,51.871608,0,0,0,0,0,0,-128,0,0,0,0,0
13630,-8.784337,51.871608,0,0,0,0,0,0,-128,0,0,0,0,0


In [188]:
result_df.shape

(149952, 13)

In [186]:
def ground_truth(file_path, latlon_crs = 'epsg:4326'):
    with rasterio.open(file_path) as f:
        zz = f.read(1)
        x = np.linspace(f.bounds.left, f.bounds.right, f.shape[1])
        y = np.linspace(f.bounds.bottom, f.bounds.top, f.shape[0])
        xx, yy = np.meshgrid(x, y)
        df = pd.DataFrame({
            'x': xx.flatten(),
            'y': yy.flatten(),
            'value': zz.flatten(),
        })
        transformer = Transformer.from_crs(f.crs, latlon_crs, always_xy=False)
        df['lat'], df['lon'] = transformer.transform(xx=df.x, yy=df.y)
        df.drop(columns=['x', 'y'], inplace=True)
        df = df[['lat', 'lon', 'value']]
        file_name = file_name_from_path(file_path)
        df = df.rename(columns={"value": file_name})
        return df

In [192]:
result_df = pd.concat(df_list).fillna(0.0)
result_df

,lat,lon,B02_10m,B03_10m,B04_10m,B08_10m,B11_10m,B12_10m,NDBI,NDDI,NDUI,NDVI,NDWI
0,-8.990286,51.785690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-8.988815,51.785690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-8.987344,51.785690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-8.985873,51.785690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-8.984402,51.785690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13627,-8.788750,51.871608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13628,-8.787279,51.871608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13629,-8.785808,51.871608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13630,-8.784337,51.871608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [168]:
ground_truth_df = ground_truth("data/landcover\\ground_truth.tif")
ground_truth_df

,lat,lon,ground_truth
0,-8.990286,51.785690,-128
1,-8.988815,51.785690,-128
2,-8.987344,51.785690,-128
3,-8.985873,51.785690,-128
4,-8.984402,51.785690,-128
...,...,...,...
13627,-8.788750,51.871608,-128
13628,-8.787279,51.871608,-128
13629,-8.785808,51.871608,-128
13630,-8.784337,51.871608,-128


In [190]:
ground_truth_df.shape

(13632, 3)

In [244]:
result_df.describe()

,lat,lon,B02_10m,B03_10m,B04_10m,B08_10m,B11_10m,B12_10m,ground_truth,NDBI,NDDI,NDUI,NDVI,NDWI
count,13632.000000,13632.000000,13632.000000,13632.000000,13632.000000,13632.000000,13632.000000,13632.000000,13632.000000,13632.000000,13632.0,13632.000000,13632.000000,13632.000000
mean,-8.886576,51.828649,18.251907,3.863630,11.204372,0.352113,0.311033,4.297755,-52.345511,0.007336,0.0,0.009830,0.006089,4.813820
std,0.060303,0.025063,52.580926,56.184306,47.798961,54.538419,54.644746,55.303016,73.196193,0.250781,0.0,0.262419,0.221455,5.260714
min,-8.990286,51.785690,-128.000000,-128.000000,-128.000000,-128.000000,-128.000000,-128.000000,-128.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,-8.938799,51.807170,0.000000,0.000000,0.000000,-10.000000,-10.000000,0.000000,-128.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,-8.886576,51.828649,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,0.0,0.000000,0.000000,0.000000
75%,-8.834353,51.850129,57.000000,27.000000,35.000000,14.000000,13.250000,27.000000,18.000000,0.000000,0.0,0.000000,0.000000,10.000000
max,-8.782866,51.871608,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,29.000000,13.000000,0.0,16.000000,17.000000,22.000000


In [246]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13632 entries, 0 to 13631
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lat           13632 non-null  float64
 1   lon           13632 non-null  float64
 2   B02_10m       13632 non-null  int8   
 3   B03_10m       13632 non-null  int8   
 4   B04_10m       13632 non-null  int8   
 5   B08_10m       13632 non-null  int8   
 6   B11_10m       13632 non-null  int8   
 7   B12_10m       13632 non-null  int8   
 8   ground_truth  13632 non-null  int8   
 9   NDBI          13632 non-null  int8   
 10  NDDI          13632 non-null  int8   
 11  NDUI          13632 non-null  int8   
 12  NDVI          13632 non-null  int8   
 13  NDWI          13632 non-null  int8   
dtypes: float64(2), int8(12)
memory usage: 372.9 KB


In [248]:
# Putting feature variable to X
X = result_df.drop('ground_truth',axis=1)
# Putting response variable to y
y = result_df['ground_truth']

In [250]:
X.shape

(13632, 13)

In [252]:
y.shape

(13632,)

In [254]:
# now lets split the data into train and test
from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((9542, 13), (4090, 13))

In [256]:
from sklearn.ensemble import RandomForestClassifier

classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,
                                       n_estimators=100, oob_score=True)

%%time
classifier_rf.fit(X_train, y_train)
# checking the oob score
classifier_rf.oob_score_

UsageError: Line magic function `%%time` not found.


In [258]:
# Import Python 3's print function and division
from __future__ import print_function, division

# Import GDAL, NumPy, and matplotlib
from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()